In [24]:
import pandas as pd
import numpy as np
from random import randrange, uniform
from sklearn import linear_model

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [25]:
import os
files = os.listdir('we_data/')
print(files)

['.DS_Store', 'Group_xx.csv', 'test.csv', 'train.csv', 'validation.csv']


In [26]:

train = pd.read_csv('we_data/train.csv')
test = pd.read_csv('we_data/test.csv')
validation = pd.read_csv('we_data/validation.csv')

In [6]:
advertisers = train['advertiser'].unique()
advertisers.sort()
advertisers

array([1458, 2259, 2261, 2821, 2997, 3358, 3386, 3427, 3476])

In [7]:
# Define data frames

# Train dataframe
train_df = pd.DataFrame(data=[], index=[advertisers], columns=['Imps', 'Clicks', 'Cost', 'avg CPC', 
                                                             'CTR', 'avg CPM', 'eCPC'])
# new_df.loc[3427]['Bids'] = 100

# Validation dataframe
validation_df = pd.DataFrame(data=[], index=[advertisers], columns=['Imps', 'Clicks', 'Cost', 'avg CPC', 
                                                             'CTR', 'avg CPM', 'eCPC'])
# Test dataframe
test_df = pd.DataFrame(data=[], index=[advertisers], columns=['Imps','Clicks','Cost', 'avg CPC', 
                                                             'CTR', 'avg CPM', 'eCPC'])

In [7]:
train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [8]:
test.columns

Index(['weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP', 'region',
       'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid', 'slotwidth',
       'slotheight', 'slotvisibility', 'slotformat', 'slotprice', 'creative',
       'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [9]:
# advertisers

In [10]:
# ad_group = train.groupby('advertiser')
# ad_group['advertiser'].describe()
# ad_group['advertiser'].count()

In [11]:
# train[train['advertiser'] == 1458].shape

In [12]:
train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'bidprice', 'payprice', 'keypage', 'advertiser', 'usertag'],
      dtype='object')

In [13]:
# train.groupby('advertiser')['advertiser'].count()

In [61]:
def dataexploration(dataframe, data):
    advertisers = data.groupby('advertiser')
    dataframe['Imps'] = advertisers['bidid'].nunique()
    dataframe['Clicks'] = advertisers['click'].sum()
    dataframe['Cost'] = advertisers['payprice'].sum()/1000.
    dataframe['CTR'] = (advertisers['click'].sum() / advertisers['bidid'].nunique()) * 100
    dataframe['avg CPC'] = dataframe['Cost'] / dataframe['Clicks']
#     check avg_CPM
    dataframe['avg CPM'] = dataframe['Cost'] / (advertisers['click'].count()) * 1000
    
    # check this with avg cpc?????
    dataframe['eCPC'] = dataframe['Cost'] / dataframe['Clicks']

In [23]:
train_explor = dataexploration(train_df, train)
train_df

,Imps,Clicks,Cost,avg CPC,CTR,avg CPM,eCPC
1458,492353,385,33968.736,88.230483,0.078196,68.992646,88.230483
2259,133673,43,12428.238,289.028791,0.032168,92.974931,289.028791
2261,110122,36,9873.779,274.271639,0.032691,89.662184,274.271639
2821,211366,131,18828.044,143.725527,0.061978,89.077922,143.725527
2997,49829,217,3129.267,14.420585,0.435489,62.800116,14.420585
3358,264956,202,22447.231,111.124906,0.076239,84.720599,111.124906
3386,455041,320,34931.823,109.161947,0.070323,76.766320,109.161947
3427,402806,272,30458.711,111.980555,0.067526,75.616329,111.980555
3476,310835,187,23918.779,127.907909,0.060161,76.950083,127.907909


In [25]:
train_df['CTR']

1458    0.078196
2259    0.032168
2261    0.032691
2821    0.061978
2997    0.435489
3358    0.076239
3386    0.070323
3427    0.067526
3476    0.060161
Name: CTR, dtype: float64

In [16]:
validation_explor = dataexploration(validation_df, validation)
validation_df

,Imps,Clicks,Cost,avg CPC,CTR,avg CPM,eCPC
1458,62353,49,4294.602,87.644939,0.078585,68.875627,87.644939
2259,16715,2,1568.808,784.404000,0.011965,93.856297,784.404000
2261,13550,3,1214.876,404.958667,0.022140,89.658745,404.958667
2821,26503,23,2394.900,104.126087,0.086783,90.363355,104.126087
2997,6176,26,388.784,14.953231,0.420984,62.950777,14.953231
3358,32939,23,2794.021,121.479174,0.069826,84.824099,121.479174
3386,56665,28,4350.793,155.385464,0.049413,76.780958,155.385464
3427,50183,37,3776.735,102.073919,0.073730,75.259251,102.073919
3476,38841,11,2993.751,272.159182,0.028321,77.077083,272.159182


In [17]:
# Calculate a new data frame for the test data
# test_explor = dataexploration(test_df, test)
# test_df

In [18]:
# import matplotlib.pyplot as plt

# import plotly.plotly as py
# # Learn about API authentication here: https://plot.ly/python/getting-started
# # Find your api_key here: https://plot.ly/settings/api

# y = [3, 10, 7, 5, 3, 4.5, 6, 8.1]
# N = len(y)
# x = range(N)
# width = 1/1.5
# plt.bar(x, y, width, color="blue")


# fig = plt.gcf()
# plot_url = py.plot_mpl(fig, filename='mpl-basic-bar')

In [8]:
# def evaluate_metrics(bidprices):
#     win_items = validation['bidprice'] < bidprices
#     imp = validation[win_items]
#     print('number of clicks', imp['click'].sum())
#     cost = imp['bidprice'].sum() / 1000
#     print('total cost', cost)
#     avg_CTR = (imp['click'].sum() / imp['click'].count()) * 100
# #     Check for CTR
#     print('Click through rate ', avg_CTR)
#     avg_CPC = cost / imp['click'].sum()
#     print('Average CPC', avg_CPC)
#     CPM = (cost) / (imp['click'].count()) * 1000
#     print('Cost per mille', CPM)


def evaluate_metrics(bidprices, display=True):
    win_items = validation['payprice'] < bidprices
    imp = validation[win_items]
    clicks = imp['click'].sum()

    cost = imp['payprice'].sum() / 1000

    avg_CTR = (imp['click'].sum() / imp['click'].count()) * 100
#     Check for CTR

    avg_CPC = cost / imp['click'].sum()
 
    CPM = (cost) / (imp['click'].count()) * 1000
    if display:
        print('number of clicks', clicks)
        print('total cost', cost)
        print('Click through rate ', avg_CTR)
        print('Average CPC', avg_CPC)
        print('Cost per mille', CPM)
    return clicks, cost, avg_CTR

In [21]:
########################################Constant Bidding###############################################################

In [ ]:
avg_bidprice = train['bidprice'].mean()
avg_bidprice

In [13]:
validation = pd.read_csv('we_data/validation.csv')

In [ ]:
validation.columns

In [ ]:
# Calculate metrics for constant bidding
evaluate_metrics(avg_bidprice)

In [ ]:
############################################RANDOM BIDDING ##########################################################

In [ ]:
# Random Bidding 
max_bid = validation['bidprice'].max()
min_bid = validation['bidprice'].min()
print('Max bid is', max_bid)
print('Min bid is', min_bid)

Max bid is 300
Min bid is 227


In [ ]:
rand_vec = np.random.uniform(low=min_bid, high=max_bid, size=(validation['bidprice'].count(),))
print('Length of random bid vector is',len(rand_vec))
print('The number of impressions are', validation['bidprice'].count())

In [ ]:
evaluate_metrics(rand_vec)

In [ ]:
#####################################Linear Bidding Strategy###########################################################

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:



# Average CTR (or average click-through rate) is the ratio of ad clicks to impressions in your AdWords campaigns.
# While basic CTR measures the rate of clicks on each ad, average CTR calculates the amount of clicks vs.
# impressions across your campaign, or for each individual keyword.



# lr = linear_model.LinearRegression()
# x = train[['click', 'weekday', 'hour', 'region', 'city','adexchange', 'advertiser', 'bidprice',
#           'slotwidth', 'slotheight', 'slotformat', 'slotprice']]
# x['adexchange'] = pd.to_numeric(x['adexchange'], errors='coerce')
# x['slotformat'] = pd.to_numeric(x['slotformat'], errors='coerce')
# x.fillna(0, inplace=True)
# y = x['bidprice']
# del x['bidprice']
# lr.fit(x,y)




# (x.shape, train.shape)

In [ ]:
#####################################################pCTR##############################################################

In [ ]:
# pd.get_dummies(train['useragent'], prefix='user_agent').head()
# ohe.transform(le.transform(train['useragent'])).shape

In [4]:
train.shape

(2430981, 25)

In [9]:
%%time
def preprocess(data):
    result = data['useragent'].str.split('_')
    data['os'] = np.array(result.tolist())[:,0]
    data['browser'] = np.array(result.tolist())[:,1]
    data['slotsize'] = data['slotwidth'] * data['slotheight']
    result = data['usertag'].str.split(',')
    unique_usertags = {elem
                       for elems in result.tolist()
                       for elem in elems}
    for usertag in unique_usertags:
        data['usertag_'+usertag] = result.apply(lambda x: usertag in x).astype(int)

preprocess(validation)
preprocess(test)
preprocess(train)
# train_sample.shape[0]

CPU times: user 1min 37s, sys: 4.78 s, total: 1min 42s
Wall time: 1min 43s


In [10]:
one = pd.DataFrame() #creates a new dataframe that's empty
one = one.append(train.loc[train['click'] == 1], ignore_index = True) 
# one.sample(1793)
one.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10074,usertag_10138,usertag_10077,usertag_13042,usertag_10093,usertag_10133,usertag_10006,usertag_10117,usertag_10140,usertag_11632,usertag_10079,usertag_10059,usertag_10076,usertag_10145,usertag_10123,usertag_10125,usertag_16751,usertag_16661,usertag_13403,usertag_null,usertag_11944,usertag_10114,usertag_10142,usertag_10118,usertag_10120,usertag_13874,usertag_10110,usertag_10063,usertag_15398,usertag_13866,usertag_10146,usertag_11092,usertag_11576,usertag_11680,usertag_16593,usertag_10147,usertag_10024,usertag_13496,usertag_10116,usertag_13776,usertag_10129,usertag_10031,usertag_10126,usertag_13678,usertag_10131,usertag_13800,usertag_16617,usertag_10048,usertag_16706,usertag_10067,usertag_10148,usertag_10052,usertag_10102,usertag_11423,usertag_10115,usertag_10130,usertag_10057,usertag_11512,usertag_10083,usertag_10684,usertag_11379,usertag_16753,usertag_10111,usertag_10075,usertag_10127,usertag_14273,usertag_10149,usertag_11278,usertag_11724
0,1,1,19,f9178e5b1b562648a5f63d36e423b0f26cec812d,ca22bf856b6c720994a95302f5ce71e0d6d5d416,windows_ie,114.93.5.*,79,79,3,5F1RQS9rg5scFsf,e2091196f0cdfe71fd757557004887e9,null,News_F_Width1,1000,90,0,0,80,86c2543527c86a893d4d4f68810a0416,300,80,43f4a37f42a7c5e6219e2601b26c6976,3386,"10006,10063",windows,ie,90000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,22,42c7568c0c07baa53e2ed2548d80fde568b1a4c1,9ed92c2038dfbaaf3ec37a48155e8712777cb21d,windows_ie,116.5.161.*,216,228,1,trqRTvFoMvas1m58uG,164e25d0b7c7da51805e4df49783e673,null,mm_35273380_3491320_11404023,336,280,0,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,65,bebefa5efe83beee17a3d245e7c5085b,1458,"11278,10111",windows,ie,94080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,1,5,0,a305108883429eb515cc1246bb91a1e64a537476,efbec188a3a7684e898747dd6cbe1194c7d89319,android_safari,117.136.24.*,201,202,1,trqRTvFyMNxRwmB4JKTI,9ec8ab358c331d15c67c58d8f2d26ea5,null,mm_10006435_106897_10197726,300,250,1,5,0,44966cc8da1ed40c95d59e863c8c75f0,300,101,361e128affece850342293213691a043,3386,13866,android,safari,75000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,2,19,c22254a0bad8028de4a6cdf51f502f6cf109c610,875ee65d08205ed37fd33b12de85cc86f1ae2aae,android_safari,119.22.194.*,3,4,null,null,null,null,1,320,50,FirstView,Na,160,11908,277,160,null,2997,null,android,safari,16000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,17,e42e60785c6f03678e8bdc7f5b5e3c054b902ae5,4cadd1388b905c04c641106ee3568358b655d92b,windows_chrome,14.134.245.*,368,373,1,trqRTuTJBQc7FmMs,6300ebb4dbec3c65d777696db72a7363,null,mm_14300749_2220747_9573579,300,250,1,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,108,361e128affece850342293213691a043,3386,"10063,10110",windows,chrome,75000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [360]:
# one.count()

# 1793

In [361]:
# one.count() 

In [362]:
train.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10116,usertag_11576,usertag_10077,usertag_11423,usertag_13874,usertag_null,usertag_16751,usertag_11512,usertag_13776,usertag_10125,usertag_16753,usertag_11379,usertag_10130,usertag_10074,usertag_10111,usertag_13403,usertag_14273,usertag_10114,usertag_10006,usertag_16593,usertag_13678,usertag_10117,usertag_10110,usertag_10083,usertag_11724,usertag_13042,usertag_10147,usertag_10142,usertag_11092,usertag_10093,usertag_10129,usertag_10684,usertag_10118,usertag_10048,usertag_10063,usertag_11278,usertag_10123,usertag_10079,usertag_10127,usertag_10133,usertag_10120,usertag_11632,usertag_10102,usertag_10031,usertag_10145,usertag_11944,usertag_10148,usertag_10075,usertag_10140,usertag_10052,usertag_10149,usertag_10067,usertag_11680,usertag_10115,usertag_16661,usertag_16617,usertag_10131,usertag_10024,usertag_10146,usertag_10126,usertag_16706,usertag_10138,usertag_13496,usertag_10057,usertag_10059,usertag_10076,usertag_15398,usertag_13866,usertag_13800
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2,trqRTvKaXTKfgg24JKTI,6447a7dfa30fe6eb410c91860e7c9b45,null,2015392487,200,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,null,windows,ie,40000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1,20fc675468712705dbf5d3eda94126da,3ddf173a94bd23c326683b6373c75dd4,null,mm_10982364_973726_8930541,300,250,FourthView,Na,0,10722,294,23,null,2821,null,windows,chrome,75000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2,trqRTJn7O95I1mKYUV,625d1b5916ea925332c7b326c0574cfa,null,1720123646,250,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",windows,ie,62500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,null,mm_10027070_118039_10308280,160,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",windows,ie,96000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2,13625cb070ffb306b425cd803c4b7ab4,4199d1227c511fc776b76594dabff9f8,null,1120200150,728,90,OtherView,Na,133,7330,277,133,null,2259,null,windows,ie,65520,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
zero = pd.DataFrame() #creates a new dataframe that's empty
zero = zero.append(train.loc[train['click'] == 0], ignore_index = True) 
zero.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10074,usertag_10138,usertag_10077,usertag_13042,usertag_10093,usertag_10133,usertag_10006,usertag_10117,usertag_10140,usertag_11632,usertag_10079,usertag_10059,usertag_10076,usertag_10145,usertag_10123,usertag_10125,usertag_16751,usertag_16661,usertag_13403,usertag_null,usertag_11944,usertag_10114,usertag_10142,usertag_10118,usertag_10120,usertag_13874,usertag_10110,usertag_10063,usertag_15398,usertag_13866,usertag_10146,usertag_11092,usertag_11576,usertag_11680,usertag_16593,usertag_10147,usertag_10024,usertag_13496,usertag_10116,usertag_13776,usertag_10129,usertag_10031,usertag_10126,usertag_13678,usertag_10131,usertag_13800,usertag_16617,usertag_10048,usertag_16706,usertag_10067,usertag_10148,usertag_10052,usertag_10102,usertag_11423,usertag_10115,usertag_10130,usertag_10057,usertag_11512,usertag_10083,usertag_10684,usertag_11379,usertag_16753,usertag_10111,usertag_10075,usertag_10127,usertag_14273,usertag_10149,usertag_11278,usertag_11724
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2,trqRTvKaXTKfgg24JKTI,6447a7dfa30fe6eb410c91860e7c9b45,null,2015392487,200,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,null,windows,ie,40000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1,20fc675468712705dbf5d3eda94126da,3ddf173a94bd23c326683b6373c75dd4,null,mm_10982364_973726_8930541,300,250,FourthView,Na,0,10722,294,23,null,2821,null,windows,chrome,75000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2,trqRTJn7O95I1mKYUV,625d1b5916ea925332c7b326c0574cfa,null,1720123646,250,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",windows,ie,62500,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,null,mm_10027070_118039_10308280,160,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",windows,ie,96000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2,13625cb070ffb306b425cd803c4b7ab4,4199d1227c511fc776b76594dabff9f8,null,1120200150,728,90,OtherView,Na,133,7330,277,133,null,2259,null,windows,ie,65520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [224]:
# zero.count()    

# 2429188

In [77]:
from sklearn.utils import shuffle
zero = shuffle(zero)
m_sample = zero[:150000]

In [78]:
m_sample.shape
import gc;gc.collect()

81

In [211]:
# 0.2 * train.count()
train.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10116,usertag_11576,usertag_10077,usertag_11423,usertag_13874,usertag_null,usertag_16751,usertag_11512,usertag_13776,usertag_10125,usertag_16753,usertag_11379,usertag_10130,usertag_10074,usertag_10111,usertag_13403,usertag_14273,usertag_10114,usertag_10006,usertag_16593,usertag_13678,usertag_10117,usertag_10110,usertag_10083,usertag_11724,usertag_13042,usertag_10147,usertag_10142,usertag_11092,usertag_10093,usertag_10129,usertag_10684,usertag_10118,usertag_10048,usertag_10063,usertag_11278,usertag_10123,usertag_10079,usertag_10127,usertag_10133,usertag_10120,usertag_11632,usertag_10102,usertag_10031,usertag_10145,usertag_11944,usertag_10148,usertag_10075,usertag_10140,usertag_10052,usertag_10149,usertag_10067,usertag_11680,usertag_10115,usertag_16661,usertag_16617,usertag_10131,usertag_10024,usertag_10146,usertag_10126,usertag_16706,usertag_10138,usertag_13496,usertag_10057,usertag_10059,usertag_10076,usertag_15398,usertag_13866,usertag_13800
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2,trqRTvKaXTKfgg24JKTI,6447a7dfa30fe6eb410c91860e7c9b45,null,2015392487,200,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,null,windows,ie,40000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1,20fc675468712705dbf5d3eda94126da,3ddf173a94bd23c326683b6373c75dd4,null,mm_10982364_973726_8930541,300,250,FourthView,Na,0,10722,294,23,null,2821,null,windows,chrome,75000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2,trqRTJn7O95I1mKYUV,625d1b5916ea925332c7b326c0574cfa,null,1720123646,250,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110",windows,ie,62500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,null,mm_10027070_118039_10308280,160,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111",windows,ie,96000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2,13625cb070ffb306b425cd803c4b7ab4,4199d1227c511fc776b76594dabff9f8,null,1120200150,728,90,OtherView,Na,133,7330,277,133,null,2259,null,windows,ie,65520,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [234]:
one.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10116,usertag_11576,usertag_10077,usertag_11423,usertag_13874,usertag_null,usertag_16751,usertag_11512,usertag_13776,usertag_10125,usertag_16753,usertag_11379,usertag_10130,usertag_10074,usertag_10111,usertag_13403,usertag_14273,usertag_10114,usertag_10006,usertag_16593,usertag_13678,usertag_10117,usertag_10110,usertag_10083,usertag_11724,usertag_13042,usertag_10147,usertag_10142,usertag_11092,usertag_10093,usertag_10129,usertag_10684,usertag_10118,usertag_10048,usertag_10063,usertag_11278,usertag_10123,usertag_10079,usertag_10127,usertag_10133,usertag_10120,usertag_11632,usertag_10102,usertag_10031,usertag_10145,usertag_11944,usertag_10148,usertag_10075,usertag_10140,usertag_10052,usertag_10149,usertag_10067,usertag_11680,usertag_10115,usertag_16661,usertag_16617,usertag_10131,usertag_10024,usertag_10146,usertag_10126,usertag_16706,usertag_10138,usertag_13496,usertag_10057,usertag_10059,usertag_10076,usertag_15398,usertag_13866,usertag_13800
0,1,1,19,f9178e5b1b562648a5f63d36e423b0f26cec812d,ca22bf856b6c720994a95302f5ce71e0d6d5d416,windows_ie,114.93.5.*,79,79,3,5F1RQS9rg5scFsf,e2091196f0cdfe71fd757557004887e9,null,News_F_Width1,1000,90,0,0,80,86c2543527c86a893d4d4f68810a0416,300,80,43f4a37f42a7c5e6219e2601b26c6976,3386,"10006,10063",windows,ie,90000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,22,42c7568c0c07baa53e2ed2548d80fde568b1a4c1,9ed92c2038dfbaaf3ec37a48155e8712777cb21d,windows_ie,116.5.161.*,216,228,1,trqRTvFoMvas1m58uG,164e25d0b7c7da51805e4df49783e673,null,mm_35273380_3491320_11404023,336,280,0,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,65,bebefa5efe83beee17a3d245e7c5085b,1458,"11278,10111",windows,ie,94080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,5,0,a305108883429eb515cc1246bb91a1e64a537476,efbec188a3a7684e898747dd6cbe1194c7d89319,android_safari,117.136.24.*,201,202,1,trqRTvFyMNxRwmB4JKTI,9ec8ab358c331d15c67c58d8f2d26ea5,null,mm_10006435_106897_10197726,300,250,1,5,0,44966cc8da1ed40c95d59e863c8c75f0,300,101,361e128affece850342293213691a043,3386,13866,android,safari,75000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,2,19,c22254a0bad8028de4a6cdf51f502f6cf109c610,875ee65d08205ed37fd33b12de85cc86f1ae2aae,android_safari,119.22.194.*,3,4,null,null,null,null,1,320,50,FirstView,Na,160,11908,277,160,null,2997,null,android,safari,16000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2,17,e42e60785c6f03678e8bdc7f5b5e3c054b902ae5,4cadd1388b905c04c641106ee3568358b655d92b,windows_chrome,14.134.245.*,368,373,1,trqRTuTJBQc7FmMs,6300ebb4dbec3c65d777696db72a7363,null,mm_14300749_2220747_9573579,300,250,1,1,0,44966cc8da1ed40c95d59e863c8c75f0,300,108,361e128affece850342293213691a043,3386,"10063,10110",windows,chrome,75000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [79]:
now = m_sample.append(one, ignore_index=True)
now= shuffle(now)
# now.count()

In [80]:
%%time
# train_sample = train.sample(486196)
# train_sample = now.sample(101793)
# train_sample = train.sample(n=int(0.2 * (train.shape[0])))
train_sample = now
preprocess(train_sample)
print(train_sample.shape)

(151793, 97)
CPU times: user 5.14 s, sys: 242 ms, total: 5.38 s
Wall time: 5.42 s


In [274]:
with pd.option_context('display.max_columns', None):
    display(train_sample.head())

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10116,usertag_11576,usertag_10077,usertag_11423,usertag_13874,usertag_null,usertag_16751,usertag_11512,usertag_13776,usertag_10125,usertag_16753,usertag_11379,usertag_10130,usertag_10074,usertag_10111,usertag_13403,usertag_14273,usertag_10114,usertag_10006,usertag_16593,usertag_13678,usertag_10117,usertag_10110,usertag_10083,usertag_11724,usertag_13042,usertag_10147,usertag_10142,usertag_11092,usertag_10093,usertag_10129,usertag_10684,usertag_10118,usertag_10048,usertag_10063,usertag_11278,usertag_10123,usertag_10079,usertag_10127,usertag_10133,usertag_10120,usertag_11632,usertag_10102,usertag_10031,usertag_10145,usertag_11944,usertag_10148,usertag_10075,usertag_10140,usertag_10052,usertag_10149,usertag_10067,usertag_11680,usertag_10115,usertag_16661,usertag_16617,usertag_10131,usertag_10024,usertag_10146,usertag_10126,usertag_16706,usertag_10138,usertag_13496,usertag_10057,usertag_10059,usertag_10076,usertag_15398,usertag_13866,usertag_13800
54373,0,4,16,66ec0a741077000c6deaa7bdd1a63132153d1004,297c4dad9e8834b8357ace8760ec75306c3d37d5,windows_ie,60.222.239.*,15,23,3,e96dd4d612a79976fd4155437b0be59a,649fc5cdef0923f8600444e195f2d223,null,QQlive_SP_HP_bottom_Width,980,90,Na,Na,20,12632,294,43,null,2261,"10006,10110,10063",windows,ie,88200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
105713,0,0,9,67286ba0edc333174e38e6fbd4ecc35305d8b230,a07704a0365b8f1b02853ad86c1610d96846e57d,windows_ie,117.88.252.*,80,81,3,5F1RQS9rg5scFsf,5baabcb52db3381bd191f9c16da45d0,null,News_F_bottom_Width,1000,90,0,0,31,d01411218cc79bc49d2a4078c4093b76,241,33,b2e35064f3549d447edbbdfb1f707c8c,3427,"10075,10083,13042,13403,13800,10059,14273,1000...",windows,ie,90000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1
132482,0,5,13,340b537d3aabd50c7b370685c2bade082a778a56,b0c2745b6b6df1aac9ce40e22335fa64e086eeed,windows_chrome,120.128.2.*,333,334,3,5F1RQS9rg5scFsf,fdb818d7be825ef58f918613d2967c0a,null,News_F_Width1,1000,90,0,0,80,c46090c887c257b61ab1fa11baee91d8,241,88,0f951a030abdaedd733ee8d114ce2944,3427,"13403,10075,10006,10110",windows,chrome,90000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16163,0,0,12,29b0f3aad864ad948f9125b60ca7464bc0b81315,789c209f43371611010c28084c4ebda0be6a154e,windows_chrome,183.236.249.*,216,219,1,e7c43fe132354916ae317b66d0b8afd2,3d95de06bb27ee3af6afdcb1849c72f0,null,mm_12712107_1718689_9016414,336,280,FirstView,Na,0,7326,294,73,null,2259,10063,windows,chrome,94080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
166780,0,4,0,9ae70f6fd5508a4083872bb3417abb5c5ed9668d,7ee6beaefbfa5f679a392c28d965c0ce55830cc7,windows_chrome,118.122.85.*,276,277,3,5F1RQS9rg5scFsf,5c3b5fa4eba5cb6a635bdab44a9f428d,null,News_F_Width1,1000,90,0,0,80,0cd33fcb336655841d3e1441b915748d,254,84,c8758e8658b05bc3a88af7b37bdf0575,3476,"10063,10059,13403,10052,10077,10006,10111",windows,chrome,90000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [5]:
# with pd.option_context('display.max_rows', None):
#     display(pd.value_counts(train_sample['usertag'].str.split(',').apply(lambda x: list(sorted(x))[0])))

In [6]:
# with pd.option_context('display.max_rows', None):
#     display(pd.value_counts(train_sample['slotvisibility']))

In [ ]:
# algo 1
def preprocess_xgb_pctr(data):
    pctr_x = data[['weekday', 'hour', 'advertiser', 
              'slotwidth', 'slotheight', 'slotprice']]
    # region, city, slotformat, adexchange
#     for discrete in ['region', 'city', 'slotformat', 'adexchange']:
    return pctr_x
preprocess_pctr = preprocess_xgb_pctr

In [367]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import gc

# Change feature
def get_train_one_hot_columns(column_name, threshold):
    values = train_sample[column_name]
    counts = pd.value_counts(values)
    mask = values.isin(counts[counts > threshold].index)
    df = pd.get_dummies(values[mask], prefix=column_name).columns
    print(column_name, threshold, df.shape)
    return df

train_one_hot_columns = {}
train_one_hot_columns['slotvisibility'] = get_train_one_hot_columns('slotvisibility', 0)
train_one_hot_columns['slotformat'] = get_train_one_hot_columns('slotformat', 0)
train_one_hot_columns['region'] = get_train_one_hot_columns('region', 0)
train_one_hot_columns['city'] = get_train_one_hot_columns('city', 0)
train_one_hot_columns['os'] = get_train_one_hot_columns('os', 0)
train_one_hot_columns['browser'] = get_train_one_hot_columns('browser', 0)
train_one_hot_columns['slotsize'] = get_train_one_hot_columns('slotsize', 0)
train_usertag_columns = [column
                         for column in train_sample.columns
                         if column.startswith('usertag_')]

def ensure_columns(df, column_list):
    for col in np.setdiff1d(column_list, df.columns):
        df[col] = 0
    df = df[column_list]
    return df

def one_hot_encode(data, column):
    train_columns = train_one_hot_columns[column]
    one_hot_data = pd.get_dummies(data[column], prefix=column)
    one_hot_data = ensure_columns(one_hot_data, train_columns)
    return one_hot_data

def freq_ctr_encode(data, column, value=False, one_hot=True):
    ctr_data = train.groupby(column)['click'].mean()
    ctr_data.name = 'ctr'
    freq_data = train.groupby(column)['click'].count()
    freq_data.name = 'freq'
    feat_data = pd.concat([ctr_data, freq_data], axis=1)
#     feat_data = pd.concat([ctr_data], axis=1)
    result = data.join(feat_data, on=column, lsuffix='_' + column)
    result.rename(columns = {'ctr': 'ctr_'+column, 'freq': 'freq_'+column}, inplace=True)
    result.rename(columns = {'ctr': 'ctr_'+column}, inplace=True)
    result_cols = ['ctr_'+column, 'freq_'+column]
#     result_cols = ['ctr_'+column]
    if not value:
        return result[result_cols]
    if one_hot:
        return pd.concat([result[result_cols], 
                          one_hot_encode(data, column)
                         ], axis=1)
    return result[result_cols + [column]]



def preprocess_lg_pctr(data):
    gc.collect()
    pctr_x = data[['weekday', 'hour', 'adexchange', 'advertiser', 
              'slotwidth', 'slotheight']]
    
    pctr_x.loc[:,'adexchange'] = pd.to_numeric(pctr_x['adexchange'], errors='coerce')

    usertag_df = pd.concat([data[column]
                            for column in data.columns
                            if column.startswith('usertag_')],
                          axis=1)
    usertag_df = ensure_columns(usertag_df, train_usertag_columns)

    pctr_x = pd.concat([pctr_x, usertag_df], axis=1)

    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotvisibility')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotformat')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'advertiser')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotsize')], axis=1)
    
    
    
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'weekday')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'hour')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'adexchange')], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotvisibility')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotformat')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotsize')], axis=1)


    # multiply by 1 to make True/False into 1/0
    pctr_x.loc[:, 'floorprice0'] = 1*(data['slotprice'] == 0)
    pctr_x.loc[:, 'floorprice1'] = 1*((1 <= data['slotprice']) & (data['slotprice'] <= 10))
    pctr_x.loc[:, 'floorprice2'] = 1*((11 <= data['slotprice']) & (data['slotprice'] <= 50))
    pctr_x.loc[:, 'floorprice3'] = 1*((51 <= data['slotprice']) & (data['slotprice'] <= 100))
    pctr_x.loc[:, 'floorprice4'] = 1*((101 <= data['slotprice']))

    pctr_x.fillna(0, inplace=True)
    return pctr_x

# gc.collect()
preprocess_pctr = preprocess_lg_pctr

slotvisibility 0 (11,)
slotformat 0 (4,)
region 0 (35,)
city 0 (368,)
os 0 (6,)
browser 0 (8,)
slotsize 0 (26,)


In [87]:
# new = preprocess_pctr(train_sample)

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [81]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import gc

# algo 2
def get_train_one_hot_columns(column_name, threshold):
    values = train_sample[column_name]
    counts = pd.value_counts(values)
    mask = values.isin(counts[counts > threshold].index)
    df = pd.get_dummies(values[mask], prefix=column_name).columns
    print(column_name, threshold, df.shape)
    return df

train_one_hot_columns = {}
train_one_hot_columns['adexchange'] = get_train_one_hot_columns('adexchange', 0)
train_one_hot_columns['slotvisibility'] = get_train_one_hot_columns('slotvisibility', 0)
train_one_hot_columns['slotformat'] = get_train_one_hot_columns('slotformat', 0)
# train_one_hot_columns['region'] = get_train_one_hot_columns('region', 3000)
train_one_hot_columns['region'] = get_train_one_hot_columns('region', 0)
# train_one_hot_columns['city'] = get_train_one_hot_columns('city', 1000)
train_one_hot_columns['city'] = get_train_one_hot_columns('city', 0)
train_one_hot_columns['os'] = get_train_one_hot_columns('os', 0)
train_one_hot_columns['browser'] = get_train_one_hot_columns('browser', 0)
train_one_hot_columns['slotsize'] = get_train_one_hot_columns('slotsize', 0)
train_usertag_columns = [column
                         for column in train_sample.columns
                         if column.startswith('usertag_')]

def ensure_columns(df, column_list):
    for col in np.setdiff1d(column_list, df.columns):
        df[col] = 0
    df = df[column_list]
    return df

def one_hot_encode(data, column):
    train_columns = train_one_hot_columns[column]
    one_hot_data = pd.get_dummies(data[column], prefix=column)
    one_hot_data = ensure_columns(one_hot_data, train_columns)
    return one_hot_data

def preprocess_lg_pctr(data):
    gc.collect()
    pctr_x = data[['weekday', 'hour', 'adexchange', 'advertiser', 
              'slotwidth', 'slotheight']]
    
    
    pctr_x.loc[:,'adexchange'] = pd.to_numeric(pctr_x['adexchange'], errors='coerce')

    usertag_df = pd.concat([data[column]
                            for column in data.columns
                            if column.startswith('usertag_')],
                          axis=1)
    usertag_df = ensure_columns(usertag_df, train_usertag_columns)

    pctr_x = pd.concat([pctr_x, usertag_df], axis=1)

#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'adexchange')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotvisibility')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotformat')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotsize')], axis=1)

    # multiply by 1 to make True/False into 1/0
    pctr_x.loc[:, 'floorprice0'] = 1*(data['slotprice'] == 0)
    pctr_x.loc[:, 'floorprice1'] = 1*((1 <= data['slotprice']) & (data['slotprice'] <= 10))
    pctr_x.loc[:, 'floorprice2'] = 1*((11 <= data['slotprice']) & (data['slotprice'] <= 50))
    pctr_x.loc[:, 'floorprice3'] = 1*((51 <= data['slotprice']) & (data['slotprice'] <= 100))
    pctr_x.loc[:, 'floorprice4'] = 1*((101 <= data['slotprice']))

    pctr_x.fillna(0, inplace=True)
    return pctr_x

# gc.collect()
preprocess_pctr = preprocess_lg_pctr

adexchange 0 (5,)
slotvisibility 0 (11,)
slotformat 0 (4,)
region 0 (35,)
city 0 (368,)
os 0 (6,)
browser 0 (8,)
slotsize 0 (25,)


In [284]:
import gc

# algo 3
def freq_ctr_encode(data, column, value=False, one_hot=True):
    ctr_data = train.groupby(column)['click'].mean()
    ctr_data.name = 'ctr'
    freq_data = train.groupby(column)['click'].count()
    freq_data.name = 'freq'
#     ecpc_data = train.groupby(column)['payprice'].sum()/(1000*train_sample.groupby(column)['click'].sum())
#     ecpc_data[~np.isfinite(ecpc_data)] = 0
#     ecpc_data.name = 'ecpc'
    feat_data = pd.concat([ctr_data, freq_data], axis=1)
    result = data.join(feat_data, on=column, lsuffix='_' + column)
#     result.rename(columns = {'ctr': 'ctr_'+column, 'freq': 'freq_'+column, 'ecpc': 'ecpc_'+column}, inplace=True)
#     result_cols = ['ctr_'+column, 'freq_'+column, 'ecpc_'+column]
    result.rename(columns = {'ctr': 'ctr_'+column, 'freq': 'freq_'+column}, inplace=True)
    result_cols = ['ctr_'+column, 'freq_'+column]
    if not value:
        return result[result_cols]
    if one_hot:
        return pd.concat([result[result_cols], 
                          one_hot_encode(data, column)
                         ], axis=1)
    return result[result_cols + [column]]

def freq_ctr_encode_tag(data, tag):
    mask = train[tag] == 1
    tag_data = train[mask]
    ctr_data = tag_data['click'].mean()
#     freq_data = tag_data['click'].count()
#     ecpc_data = tag_data['payprice'].sum()/(1000*tag_data['click'].sum())
#     if np.any(~np.isfinite(ecpc_data)):
#         ecpc_data = 0
#     result = pd.DataFrame({'ctr_'+tag: data[tag], 'freq_'+tag: data[tag], 'ecpc_'+tag: data[tag]})
    result = pd.DataFrame({'ctr_'+tag: data[tag]})
    result.loc[mask, 'ctr_'+tag] = ctr_data
#     result.loc[mask, 'freq_'+tag] = freq_data
#     result.loc[mask, 'ecpc_'+tag] = ecpc_data
    return result

def preprocess_super_pctr(data):
    gc.collect()
    pctr_x = data[['weekday', 'hour', 'adexchange', 'advertiser','slotwidth', 'slotheight', 'slotprice']]
    pctr_x.loc[:,'adexchange'] = pd.to_numeric(pctr_x['adexchange'], errors='coerce')
    usertag_df = pd.concat([data[column]
                            for column in data.columns
                            if column.startswith('usertag_')],
                          axis=1)
    usertag_df = ensure_columns(usertag_df, train_usertag_columns)

    pctr_x = pd.concat([pctr_x] + [freq_ctr_encode_tag(usertag_df, column)
                                   for column in usertag_df.columns], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'weekday')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'hour')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'advertiser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'adexchange')], axis=1)

    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotvisibility')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotformat')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'city')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'region')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'os')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'browser')], axis=1)
    pctr_x = pd.concat([pctr_x, freq_ctr_encode(data, 'slotsize')], axis=1)

#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotvisibility')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotformat')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'city')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'region')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'os')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'browser')], axis=1)
#     pctr_x = pd.concat([pctr_x, one_hot_encode(data, 'slotsize')], axis=1)

    # multiply by 1 to make True/False into 1/0
#     train_sample[(train_sample['slotprice'] < 100) & (train_sample['slotprice'] > 0)]['slotprice'].plot.kde()
#     plt.show()

#     for colnum, span in enumerate([(None, 0), (1, 10), (11, 25), (26, 60), (61, 75), (76, 100), (101, None)]):
#         pctr_x.loc[:, 'floorprice' + str(colnum)] = 0
#         if span[0] is None:
#             data_mask = (data['slotprice'] <= span[0])
#             train_mask = (train['slotprice'] <= span[0])
#         elif span[1] is None:
#             data_mask = (data['slotprice'] >= span[1])
#             train_mask = (train['slotprice'] >= span[1])
#         else:
#             data_mask = ((span[0] <= data['slotprice']) & (data['slotprice'] <= span[1]))
#             train_mask = ((span[0] <= train['slotprice']) & (train['slotprice'] <= span[1]))
#         pctr_x.loc[data_mask, 'floorprice0'] = train[train_mask]['click'].mean()

#     pctr_x.loc[:, 'floorprice0'] = 1*(data['slotprice'] <= 0)
#     pctr_x.loc[:, 'floorprice1'] = 1*((1 <= data['slotprice']) & (data['slotprice'] <= 10))
#     pctr_x.loc[:, 'floorprice2'] = 1*((11 <= data['slotprice']) & (data['slotprice'] <= 25))
#     pctr_x.loc[:, 'floorprice3'] = 1*((26 <= data['slotprice']) & (data['slotprice'] <= 60))
#     pctr_x.loc[:, 'floorprice4'] = 1*((61 <= data['slotprice']) & (data['slotprice'] <= 75))
#     pctr_x.loc[:, 'floorprice5'] = 1*((76 <= data['slotprice']) & (data['slotprice'] <= 100))
#     pctr_x.loc[:, 'floorprice6'] = 1*((101 <= data['slotprice']))

    pctr_x.fillna(0, inplace=True)
    return pctr_x

# gc.collect()
preprocess_pctr = preprocess_super_pctr

In [17]:
train_sample.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag,os,browser,slotsize,usertag_10120,usertag_16706,usertag_13678,usertag_10077,usertag_16753,usertag_13496,usertag_11724,usertag_11680,usertag_10123,usertag_13874,usertag_10052,usertag_11379,usertag_10114,usertag_16617,usertag_11632,usertag_10149,usertag_10116,usertag_10126,usertag_10063,usertag_10083,usertag_10148,usertag_10129,usertag_10140,usertag_10145,usertag_10031,usertag_10110,usertag_10093,usertag_10075,usertag_14273,usertag_10684,usertag_10127,usertag_10067,usertag_16751,usertag_13866,usertag_10146,usertag_10125,usertag_10138,usertag_10006,usertag_13776,usertag_10079,usertag_10131,usertag_11576,usertag_10117,usertag_10130,usertag_10076,usertag_15398,usertag_10111,usertag_11423,usertag_16593,usertag_16661,usertag_10048,usertag_13403,usertag_10057,usertag_11944,usertag_13800,usertag_10074,usertag_13042,usertag_10118,usertag_11092,usertag_10133,usertag_10142,usertag_10115,usertag_11512,usertag_10059,usertag_10102,usertag_10024,usertag_10147,usertag_null,usertag_11278
94440,0,5,11,0a19118fa5a441b86976706a82afdb23191b5ffd,6ff1e9bbdba24b292c37f71d99e4702bf24f0daf,windows_chrome,113.229.55.*,40,54,2,kJa-GDFRGql7gspy,3512c349cc3097a60ecb42be0e94cb9e,null,125237984,728,90,2,0,5,48f2e9ba15708c0146bda5e1dd653caa,300,65,bebefa5efe83beee17a3d245e7c5085b,1458,"10063,10083,10006,10110",windows,chrome,65520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
74964,0,5,14,244e7d8b848f5f7b06fbc013bb6dc3b252103a4c,fa1888def9d516dfef337b4ab2f26cc0fd741195,windows_chrome,183.32.88.*,216,234,1,e7c43fe132354916ae317b66d0b8afd2,1d575006ec00949673f9a30a041cf0ed,null,mm_12712107_1718689_9016414,336,280,FirstView,Na,0,7326,294,151,null,2259,null,windows,chrome,94080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
63228,0,4,11,83b27c766b7f9e2b3ebac00e7daca876e07d06c4,3db42e59099bbbe899b7ce26f879588eb601ae3b,windows_chrome,1.61.101.*,65,77,3,31xSTvprdN1RFt,f04fb9d06f4d9dfb35bfbafd701751c6,null,ALLINONE_F_Width2,1000,90,0,0,50,a10c31a8ff5f42930b4c34035e523886,241,55,d29e59bf0f7f8243858b8183f14d4412,3358,"10006,10063,13403,10031",windows,chrome,90000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36182,0,3,20,1cb64e5a24b5e8b619b3cb78411fac41b914892b,4d552a12cc7d49c601ee3f90db82e54c71c971fa,windows_ie,113.226.220.*,40,42,1,DFpETuxoGQdcFNKbuKz,4a77427681e92df4f35504e1d2718a16,null,mm_10024662_3445902_11178345,336,280,2,1,0,77819d3e0b3467fe5c7b16d68ad923a1,300,16,bebefa5efe83beee17a3d245e7c5085b,1458,"10083,10063,10006,10110",windows,ie,94080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6677,0,5,11,2a5dc2201bdf871c2132c015f3b906d80f7f1855,a13e696310838013bdc645aadbfa609bd0e2b130,windows_chrome,14.145.188.*,216,217,3,b65f0a1b32379143518d4d25999d19f3,6d083385bb7e3f192636df59081e0cad,null,discuz_1027125_001,120,240,Na,Na,40,7314,294,40,null,2259,"10059,10684,14273,10006,10111,10063",windows,chrome,28800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [42]:
# preprocess_pctr(train_sample).head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,weekday,hour,adexchange,advertiser,slotwidth,slotheight,slotprice,ctr_usertag_10116,ctr_usertag_11576,ctr_usertag_10077,ctr_usertag_11423,ctr_usertag_13874,ctr_usertag_null,ctr_usertag_16751,ctr_usertag_11512,ctr_usertag_13776,ctr_usertag_10125,ctr_usertag_16753,ctr_usertag_11379,ctr_usertag_10130,ctr_usertag_10074,ctr_usertag_10111,ctr_usertag_13403,ctr_usertag_14273,ctr_usertag_10114,ctr_usertag_10006,ctr_usertag_16593,ctr_usertag_13678,ctr_usertag_10117,ctr_usertag_10110,ctr_usertag_10083,ctr_usertag_11724,ctr_usertag_13042,ctr_usertag_10147,ctr_usertag_10142,ctr_usertag_11092,ctr_usertag_10093,ctr_usertag_10129,ctr_usertag_10684,ctr_usertag_10118,ctr_usertag_10048,ctr_usertag_10063,ctr_usertag_11278,ctr_usertag_10123,ctr_usertag_10079,ctr_usertag_10127,ctr_usertag_10133,ctr_usertag_10120,ctr_usertag_11632,ctr_usertag_10102,ctr_usertag_10031,ctr_usertag_10145,ctr_usertag_11944,ctr_usertag_10148,ctr_usertag_10075,ctr_usertag_10140,ctr_usertag_10052,ctr_usertag_10149,ctr_usertag_10067,ctr_usertag_11680,ctr_usertag_10115,ctr_usertag_16661,ctr_usertag_16617,ctr_usertag_10131,ctr_usertag_10024,ctr_usertag_10146,ctr_usertag_10126,ctr_usertag_16706,ctr_usertag_10138,ctr_usertag_13496,ctr_usertag_10057,ctr_usertag_10059,ctr_usertag_10076,ctr_usertag_15398,ctr_usertag_13866,ctr_usertag_13800,ctr_weekday,freq_weekday,ctr_hour,freq_hour,ctr_advertiser,freq_advertiser,ctr_adexchange,freq_adexchange,ctr_slotvisibility,freq_slotvisibility,ctr_slotformat,freq_slotformat,ctr_city,freq_city,ctr_region,freq_region,ctr_os,freq_os,ctr_browser,freq_browser,ctr_slotsize,freq_slotsize
2391760,2,3,3.0,3386,1000,90,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000571,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000965,296438,0.000453,33126,0.000703,455041,0.000688,819458,0.000693,1006009,0.000617,1331454,0.000513,109078,0.000577,407378,0.000572,2314896,0.000646,1461700,0.000752,553283
1383443,1,14,4.0,2821,728,90,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000532,0.0,0.000537,0.0,0.0,0.000000,0.000588,0.000000,0.0,0.000000,0.0,0.0,0.000418,0.000000,0.0,0.001993,0.0,0.000000,0.000571,0.0,0.0,0.000545,0.0,0.000507,0.000338,0.0,0.0,0.000536,0.0,0.0,0.0,0.000000,0.0,0.000632,0.0,0.0,0.0,0.000508,0.0,0.0,0.0,0.000000,0.000502,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000455,0.000676,368164,0.000610,132772,0.000620,211366,0.000475,79995,0.001831,142023,0.000846,504990,0.001116,3585,0.000768,108104,0.000572,2314896,0.000450,797992,0.000613,372197
521699,3,8,1.0,3476,300,250,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000494,0.000892,0.000000,0.000000,0.0,0.000537,0.0,0.0,0.000000,0.000000,0.000522,0.0,0.000000,0.0,0.0,0.000000,0.000515,0.0,0.000000,0.0,0.000000,0.000571,0.0,0.0,0.000545,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000511,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000534,0.000000,0.000000,0.0,0.0,0.000675,0.000511,0.000494,0.000461,0.0,0.000000,0.000455,0.000833,339627,0.000448,69197,0.000602,310835,0.000819,718271,0.000693,1006009,0.000743,582564,0.001422,4921,0.001110,63055,0.000572,2314896,0.000450,797992,0.000828,619285
534394,4,17,2.0,2261,120,600,180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000461,0.0,0.0,0.000000,0.000000,0.000554,0.000000,0.0,0.000537,0.0,0.0,0.000157,0.000588,0.000522,0.0,0.001996,0.0,0.0,0.000000,0.000515,0.0,0.001993,0.0,0.000532,0.000571,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000536,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000534,0.000000,0.000445,0.0,0.0,0.000000,0.000000,0.000494,0.000000,0.0,0.000655,0.000000,0.000810,326018,0.001017,1435

In [62]:
# CTR: train_sample.groupby('region')['click'].mean()
# frequency: train_sample.groupby('region')['click'].count()
preprocess_pctr

<function __main__.preprocess_lg_pctr>

In [22]:
# result = train_sample['usertag'].str.split(',')
# result = np.array(result.tolist())
# # data['os'] = np.array(result.tolist())[:,0]
# # data['browser'] = np.array(result.tolist())[:,1]

In [82]:
len(train_sample['slotid'].unique())

13973

In [27]:

# bid = base bid×pCT R/avgCT R

In [40]:
# pctr_train_y = train_sample['click']
# pctr_train_x = preprocess_lg_pctr(train_sample)
# lg = linear_model.LogisticRegression()
# lg.fit(pctr_train_x, pctr_train_y)

In [41]:
# pctr_train_x.head()

In [42]:
# logistic_pctr = lg.predict_proba(pctr_train_x)
# avgCTR = train_sample['click'].mean()
# lr = linear_model.LinearRegression(fit_intercept=False)
# lr_feat_x = logistic_pctr[:,1]/avgCTR
# lr_feat_x = lr_feat_x.reshape(-1, 1)
# print (lr_feat_x.shape)
# print(train_sample['bidprice'].shape)
# lr.fit(lr_feat_x, train_sample['bidprice'])

In [43]:



# lr.coef_

In [44]:
# logistic_pctr[:,1]

In [45]:
# # bid = base bid×pCT R/avgCT R

# pctr_valid_x = preprocess_lg_pctr(validation)
# logistic_pctr = lg.predict_proba(pctr_valid_x)
# base_bid = lr.coef_ * 1.2
# logistic_bids = base_bid * logistic_pctr[:,1]/avgCTR
# evaluate_metrics(logistic_bids)

In [46]:
# # bid = base bid×pCT R/avgCT R

# pctr_valid_x = preprocess_lg_pctr(validation)
# logistic_pctr = lg.predict_proba(pctr_valid_x)
# base_bid = lr.coef_ * 1.2
# logistic_bids = base_bid * logistic_pctr[:,1]/avgCTR
# evaluate_metrics(logistic_bids)

In [47]:
# for basebid in range(20,100):
#     sum = 6250 * 1000
#     clicks = 0
#     for pay,click,pCTR in zip(validation["payprice"],validation['click'],logistic_pctr[:,1]):
#         sum_sub = sum - pay
#         bidprice = basebid * pCTR / avgCTR
#         if bidprice-pay > 0 and sum_sub >= 0:
#             sum = sum_sub
#             clicks += click
#     print(basebid, clicks, sum)



In [48]:
# pctr_test_x = preprocess_lg_pctr(test)
# logistic_pctr = lg.predict_proba(pctr_test_x)
# base_bid = lr.coef_ * 1.2

# logistic_bids = base_bid * logistic_pctr[:,1]/avgCTR


In [50]:
# lr.coef_

In [51]:
#####################################################Cv################################################################

In [130]:
# from sklearn.model_selection import cross_val_score


In [131]:
###################################çççççççççççççççççççççççççççç########################################################

In [82]:
%%time

pctr_train_x = preprocess_pctr(train_sample)
assert(np.all(np.isfinite(pctr_train_x)))
print(pctr_train_x.shape)
pctr_train_x.head()

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(151793, 154)
CPU times: user 1.14 s, sys: 978 ms, total: 2.12 s
Wall time: 2.13 s


In [102]:
# pctr_train_x.head()

,weekday,hour,adexchange,advertiser,slotwidth,slotheight,usertag_10116,usertag_11576,usertag_10077,usertag_11423,usertag_13874,usertag_null,usertag_16751,usertag_11512,usertag_13776,usertag_10125,usertag_16753,usertag_11379,usertag_10130,usertag_10074,usertag_10111,usertag_13403,usertag_14273,usertag_10114,usertag_10006,usertag_16593,usertag_13678,usertag_10117,usertag_10110,usertag_10083,usertag_11724,usertag_13042,usertag_10147,usertag_10142,usertag_11092,usertag_10093,usertag_10129,usertag_10684,usertag_10118,usertag_10048,usertag_10063,usertag_11278,usertag_10123,usertag_10079,usertag_10127,usertag_10133,usertag_10120,usertag_11632,usertag_10102,usertag_10031,usertag_10145,usertag_11944,usertag_10148,usertag_10075,usertag_10140,usertag_10052,usertag_10149,usertag_10067,usertag_11680,usertag_10115,usertag_16661,usertag_16617,usertag_10131,usertag_10024,usertag_10146,usertag_10126,usertag_16706,usertag_10138,usertag_13496,usertag_10057,usertag_10059,usertag_10076,usertag_15398,usertag_13866,usertag_13800,slotformat_0,slotformat_1,slotformat_5,slotformat_Na,os_android,os_ios,os_linux,os_mac,os_other,os_windows,ctr_advertiser,browser_chrome,browser_firefox,browser_ie,browser_maxthon,browser_opera,browser_other,browser_safari,browser_sogou,browser_theworld,floorprice0,floorprice1,floorprice2,floorprice3,floorprice4
2391760,2,3,3.0,3386,1000,90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0.000703,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1383443,1,14,4.0,2821,728,90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0.000620,1,0,0,0,0,0,0,0,0,0,0,1,0,0
521699,3,8,1.0,3476,300,250,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0.000602,1,0,0,0,0,0,0,0,0,1,0,0,0,0
534394,4,17,2.0,2261,120,600,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0.000327,1,0,0,0,0,0,0,0,0,0,0,0,0,1
331468,0,16,3.0,2259,1000,90,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.000322,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [20]:
train_sample.shape

(101793, 97)

In [91]:
%%time
import xgboost as xgb

# algorithm 1:
# m = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.05, n_jobs=4)
# algorithm 2:
m = xgb.XGBClassifier(max_depth=8, n_estimators=100, learning_rate=0.1, n_jobs=4)
# algorithm 3:
# auc 80
# m = linear_model.LogisticRegression(C=0.1, penalty='l1')
# m = neural_network.MLPClassifier(verbose=True, hidden_layer_sizes=(50, 50))
# m = xgb.XGBClassifier(max_depth=5, n_estimators=100, learning_rate=0.5, n_jobs=4)

pctr_train_y = train_sample['click']
m.fit(pctr_train_x, pctr_train_y)
xgb_pctr = m.predict_proba(pctr_train_x)

CPU times: user 2min 41s, sys: 2 s, total: 2min 43s
Wall time: 46.7 s


In [12]:
feat=dict(zip(pctr_train_x.columns,m.coef_.reshape(-1).tolist()))
used_feat = {k: v for k, v in feat.items() if v > 0}
sorted_feat = sorted(used_feat.items(), key=operator.itemgetter(1), reverse=True)

plt.figure(figsize=(20,5))
plt.bar(range(len(used_feat)), [f[1] for f in sorted_feat], align='center')
plt.xticks(range(len(used_feat)), [f[0] for f in sorted_feat], rotation='vertical')
plt.gca().set_yscale('log')
plt.show()
print('unused', len([k for k,v in feat.items() if v == 0]))
print('total', len(feat))

AttributeError: 'XGBClassifier' object has no attribute 'coef_'

In [93]:
%%time
avgCTR = train_sample['click'].mean()
lr = linear_model.LinearRegression(fit_intercept=False)
lr_feat_x = xgb_pctr[:,1]/avgCTR
lr_feat_x = lr_feat_x.reshape(-1, 1)
print (lr_feat_x.shape)
print(train_sample['bidprice'].shape)
lr.fit(lr_feat_x, train_sample['bidprice'])

(151793, 1)
(151793,)
CPU times: user 6.17 ms, sys: 2.56 ms, total: 8.73 ms
Wall time: 7.77 ms


In [303]:
# base_bid = lr.coef_ * 1.85
# print("uhiedh", 80.55 / lr.coef_ )

In [304]:
# pctr_valid_x = preprocess_lg_pctr(validation)
# xgb_valid_pctr = m.predict_proba(pctr_valid_x)
# base_bid = lr.coef_ * 1.84
# xgb_bids = base_bid * xgb_valid_pctr[:,1]/avgCTR
# evaluate_metrics(xgb_bids)

In [344]:
# validation.count()
# 303925
# train.count()
# 2430981

In [372]:
# onev = pd.DataFrame() #creates a new dataframe that's empty
# onev = onev.append(validation.loc[validation['click'] == 1], ignore_index = True) 
# onev.sample(202)
# zerov = pd.DataFrame() #creates a new dataframe that's empty
# zerov = zerov.append(validation.loc[validation['click'] == 0], ignore_index = True) 
# v_sample = zerov.sample(12524)


# nowv = v_sample.append(onev, ignore_index=True)
# from sklearn.utils import shuffle
# nowv= shuffle(nowv)
# nowv.count()




In [94]:
%%time
validation= shuffle(validation)
pctr_valid_x = preprocess_pctr(validation)

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 3.34 s, sys: 2.65 s, total: 5.99 s
Wall time: 6.18 s


In [95]:
%%time
xgb_valid_pctr = m.predict_proba(pctr_valid_x)

CPU times: user 3.29 s, sys: 1.05 s, total: 4.33 s
Wall time: 2.81 s


In [97]:
%%time
budget = 6250
best_scale = None
best_clicks = None

for scale in np.linspace(7,100.,1000):
    base_bid = lr.coef_ * scale
    xgb_bids = base_bid * xgb_valid_pctr[:,1]/avgCTR
    clicks, cost, avg_CTR = evaluate_metrics(xgb_bids, display=False)
#     print('clicks', clicks, scale, base_bid)
    if best_clicks is None or (clicks > best_clicks and cost < budget):
        best_clicks = clicks
        best_scale = scale
        print('new best clicks: num clicks ', best_clicks, ' scale ', best_scale, ' base bid ', lr.coef_, ' cost ', cost)

new best clicks: num clicks  144  scale  7.0  base bid  [ 10.02639384]  cost  2883.318
new best clicks: num clicks  146  scale  7.27927927928  base bid  [ 10.02639384]  cost  3026.3
new best clicks: num clicks  147  scale  7.55855855856  base bid  [ 10.02639384]  cost  3171.611
new best clicks: num clicks  148  scale  7.93093093093  base bid  [ 10.02639384]  cost  3358.285
new best clicks: num clicks  150  scale  8.02402402402  base bid  [ 10.02639384]  cost  3401.646
new best clicks: num clicks  151  scale  8.11711711712  base bid  [ 10.02639384]  cost  3444.031
new best clicks: num clicks  152  scale  8.3963963964  base bid  [ 10.02639384]  cost  3583.859
new best clicks: num clicks  153  scale  8.76876876877  base bid  [ 10.02639384]  cost  3769.779
new best clicks: num clicks  154  scale  8.95495495495  base bid  [ 10.02639384]  cost  3855.985
new best clicks: num clicks  155  scale  9.42042042042  base bid  [ 10.02639384]  cost  4079.285
new best clicks: num clicks  157  scale  9.

KeyboardInterrupt: 

In [99]:
%%time
pctr_test_x = preprocess_lg_pctr(test)
xgb_test_pctr = m.predict_proba(pctr_test_x)
base_bid = 14.2612612613  * 10.02639384
xgb_bids = base_bid * xgb_test_pctr[:,1]/avgCTR
# evaluate_metrics(xgb_bids)
xgb_bids.shape

/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


CPU times: user 5.96 s, sys: 4.16 s, total: 10.1 s
Wall time: 9.05 s


In [ ]:
# csv_df = pd.DataFrame()
# csv_df['bidid'] = test['bidid']
# csv_df['bidprice'] = xgb_bids
# csv_df.to_csv('testing_bidding_price.csv')

In [ ]:
#######################################################################################################################


In [23]:
# test['bidid'].shape

(303375,)

In [1]:
# import xgboost as xgb
# m = xgb.XGBClassifier(max_depth=4, n_estimators=100, n_jobs=2)
# pctr_train_y = train['payprice']
# pctr_train_x = preprocess_lg_pctr(train)
# m.fit(pctr_train_x, pctr_train_y)
# xgb_pctr = m.predict(pctr_train_x)

In [ ]:
# xgb_pctr.head()

In [100]:
csv_df = pd.DataFrame()
csv_df['bidid'] = test['bidid']
csv_df['bidprice'] = xgb_bids
csv_df.to_csv('testing_bidding_price_xgb504171.csv')

In [3]:
file1 = pd.read_csv('testing_bidding_price_xgb504171.csv')
file1.head()

,Unnamed: 0,bidid,bidprice
0,0,366c563de7d90feb9d4dab53e795a93fb3157387,104.628502
1,1,29167d4caa719788b5a342dbaa25151d53121f80,789.159668
2,2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,14.607371
3,3,844c2da00d45315f20b748ec131c26ee99a7cbc7,50.131500
4,4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,151.856186


In [101]:
# PSL16u68d1n4
# Linear regression(xgboost for pctr) bidding strategy


!curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_bidding_price_xgb504171.csv'


{"ranking": 4, "group": "20", "result": {"impressions": 131011, "cost": 6230.157000002232, "clicks": 179, "ctr": 0.0013662974864706017, "cpc": 34.805346368727555}, "daily submission limit": 3, "today tried times": 2, "best result": {"impressions": 131011, "cost": 6230.157000002232, "clicks": 179, "ctr": 0.0013662974864706017, "cpc": 34.805346368727555}}


In [28]:
# PSL16u68d1n4
# Linear regression(xgboost for pctr) bidding strategy


!curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_bidding_price_random2-25.csv'


{"ranking": 12, "group": "20", "result": {"impressions": 148433, "cost": 5678.062000002696, "clicks": 134, "ctr": 0.0009027642101150014, "cpc": 42.37359701494549}, "daily submission limit": 3, "today tried times": 2, "best result": {"impressions": 121991, "cost": 6085.894000001979, "clicks": 175, "ctr": 0.0014345320556434491, "cpc": 34.776537142868456}}


In [ ]:
testing_bidding_price_xglr2-25.csv

In [ ]:
test_bidprice_20.csv

In [10]:
f1 = pd.read_csv('test_bidprice1156.csv', index_col=0)
f2 = pd.read_csv('testing_bidding_price_xgb504171.csv', index_col=0)

In [17]:
f2.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,104.628502
1,29167d4caa719788b5a342dbaa25151d53121f80,789.159668
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,14.607371
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,50.131500
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,151.856186


In [18]:
f1.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,83.165923
1,29167d4caa719788b5a342dbaa25151d53121f80,381.991133
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,10.567278
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,72.152388
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,103.476101


In [15]:
avg = 0.4*f1['bidprice'] + 0.6*f2['bidprice']

In [16]:
# avg.to_frame().to_csv('testing_avg_final.csv')

In [27]:
csv_df = pd.DataFrame()
csv_df['bidid'] = test['bidid']
csv_df['bidprice'] = avg
csv_df.to_csv('testing_avg_final.csv')

In [28]:
f3 = pd.read_csv('testing_avg_final.csv', index_col=0)

In [29]:
f3.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,96.043470
1,29167d4caa719788b5a342dbaa25151d53121f80,626.292254
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,12.991334
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,58.939855
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,132.504152


In [30]:
# PSL16u68d1n4
# Linear regression(xgboost for pctr) bidding strategy


!curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_avg_final.csv'


{"ranking": 6, "group": "20", "result": {"impressions": 132431, "cost": 6249.999000002322, "clicks": 175, "ctr": 0.0013214428645860862, "cpc": 35.71428000001327}, "daily submission limit": 3, "today tried times": 2, "best result": {"impressions": 131011, "cost": 6230.157000002232, "clicks": 179, "ctr": 0.0013662974864706017, "cpc": 34.805346368727555}}


In [11]:
# PSL16u68d1n4
# Linear regression(xgboost for pctr) bidding strategy


!curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_avg.csv'


{"ranking": 12, "group": "20", "result": {"impressions": 138580, "cost": 6249.998000002423, "clicks": 172, "ctr": 0.0012411603405974888, "cpc": 36.337197674432694}, "daily submission limit": 3, "today tried times": 3, "best result": {"impressions": 121991, "cost": 6085.894000001979, "clicks": 175, "ctr": 0.0014345320556434491, "cpc": 34.776537142868456}}


In [30]:
# file = pd.read_csv('testing_bidding_price_xglr.csv',  index_col=0)

In [31]:
# file.head()

,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,41.693754
1,29167d4caa719788b5a342dbaa25151d53121f80,307.872479
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,78.282445
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,60.343378
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,64.381043


In [322]:
# PSL16u68d1n4
# Linear regression(xgboost for pctr) bidding strategy


!curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_bidding_price_xglr0402.csv'


{"ranking": 10, "group": "20", "result": {"impressions": 143694, "cost": 6249.998000002514, "clicks": 163, "ctr": 0.0011343549487104541, "cpc": 38.343546012285366}, "daily submission limit": 3, "today tried times": 3, "best result": {"impressions": 121991, "cost": 6085.894000001979, "clicks": 175, "ctr": 0.0014345320556434491, "cpc": 34.776537142868456}}


In [40]:
#################################################Your Best Bidding Strategy############################################

In [98]:
csv_df = pd.DataFrame()
csv_df['bidid'] = test['bidid']
csv_df['bidprice'] = logistic_bids
csv_df.to_csv('testing_bidding_price.csv')

In [99]:
# PSL16u68d1n4
# Linear regression(logistic regression for pctr) bidding strategy
# !curl http://deepmining.cs.ucl.ac.uk/api/upload/wining_criteria_1/PSL16u68d1n4 -X Post -F 'file=@./testing_bidding_price.csv'


{"ranking": 4, "group": "20", "result": {"impressions": 121286, "cost": 6249.999000001894, "clicks": 90, "ctr": 0.0007420477219134937, "cpc": 69.44443333335438}, "daily submission limit": 3, "today tried times": 2, "best result": {"impressions": 121286, "cost": 6249.999000001894, "clicks": 90, "ctr": 0.0007420477219134937, "cpc": 69.44443333335438}}


In [154]:
# file = pd.read_csv('test_bidprice_20.csv',  index_col=0)
# file.head()